In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from collections import defaultdict

In [61]:
# root_path = '../lightning_logs/vp_bridge_egnn_CombinedSparseGraphDataset_2024-08-19_21_05_04.140916'
# root_path = '../lightning_logs/vp_bridge_egnn_CombinedSparseGraphDataset_2024-08-19_21_05_04.140916/ligand_based'
# root_path = '../lightning_logs/vp_bridge_egnn_CombinedSparseGraphDataset_2024-08-19_13_30_50.133260'
root_path = '../lightning_logs/vp_bridge_egnn_CombinedSparseGraphDataset_2024-08-19_13_30_50.133260/ligand_based'

In [62]:
aromatic = False
optimization = False
fn = 'reconstructed_mols' + '_aromatic_mode' if aromatic else 'reconstructed_mols'
match_fn = fn + '_matches.pkl'
score_fn = fn + '_scores.csv'
match_file = os.path.join(root_path, match_fn)
score_file = os.path.join(root_path, score_fn)
match_file

'../lightning_logs/vp_bridge_egnn_CombinedSparseGraphDataset_2024-08-19_13_30_50.133260/ligand_based/reconstructed_mols_matches.pkl'

In [63]:
with open(match_file, 'rb') as f:
    match_dict = pickle.load(f)

In [64]:
scores = pd.read_csv(score_file, index_col=0, header=0)
scores

,score
1yw8_A_rec_1yw8_a75_lig_tt_docked_2,0.600000
4f31_B_rec_3lsw_4mp_lig_tt_min_0,0.333333
5i00_A_rec_5i00_afq_lig_tt_min_0,0.642857
5eob_A_rec_3zxz_krw_lig_tt_docked_0,0.555556
4ot2_A_rec_4zbr_nps_lig_tt_min_0,0.777778
...,...
4b84_A_rec_4b84_z5k_lig_tt_min_0,0.300000
2re8_A_rec_1o0b_amp_lig_tt_min_0,0.571429
2bur_A_rec_1eob_dhb_lig_tt_min_0,0.500000
5hx8_A_rec_4ivb_1j5_lig_tt_min_0,0.833333


In [65]:
score_dict = scores.to_dict()['score']
# score_dict

In [66]:
# match_dict

# select by lig

In [67]:
all_lig = match_dict.keys()
# all_lig

In [68]:
len(all_lig)

15075

In [69]:
lig_names = [lig[lig.rfind('rec')+9:lig.rfind('rec')+12] for lig in all_lig]
lig_names = set(lig_names)
len(lig_names)

999

In [70]:
# lig_names

In [71]:
lig_dict = defaultdict(lambda : [])
for lig in tqdm(all_lig):
    lig_name = lig[lig.rfind('rec')+9:lig.rfind('rec')+12]
    lig_dict[lig_name].append(lig)

100%|███████████████████████████████████████████████████████████████████| 15075/15075 [00:00<00:00, 1292659.21it/s]


In [72]:
# lig_dict

In [73]:
rev_lig_dict = {lig:lig_name for lig_name, ligs in lig_dict.items() for lig in ligs }

In [74]:
# rev_lig_dict

In [75]:
scores_by_lig = defaultdict(lambda : [])
for lig, score in tqdm(score_dict.items()):
    lig_name = rev_lig_dict[lig]
    scores_by_lig[lig_name].append(score)

# scores_by_lig

100%|███████████████████████████████████████████████████████████████████| 15075/15075 [00:00<00:00, 1694197.18it/s]


In [76]:
avg_score = {}
n_lig = {}
for lig, scores in tqdm(scores_by_lig.items()):
    avg_score[lig] = np.mean(scores)
    n_lig[lig] = len(scores)

100%|████████████████████████████████████████████████████████████████████████| 999/999 [00:00<00:00, 121136.45it/s]


In [77]:
avg_score_df = pd.DataFrame.from_dict({'score':avg_score, 'num sample':n_lig}, orient='columns')
avg_score_df

,score,num sample
a75,0.600000,8
4mp,0.333333,3
afq,0.666113,43
krw,0.550388,43
nps,0.716049,9
...,...,...
ce1,0.263158,1
cze,0.142857,1
8qp,0.545455,1
dwf,0.071429,1


In [78]:
avg_score_df.sort_values('score', ascending=False, inplace=True)

In [79]:
avg_score_df.to_csv(os.path.join(root_path, fn+'_avg_score_by_lig.csv'))

In [80]:
avg_score_df = pd.read_csv(os.path.join(root_path, fn+'_avg_score_by_lig.csv'), index_col=0, header=0)
avg_score_df

,score,num sample
hex,1.00000,3
d12,1.00000,1
d10,1.00000,16
dcd,1.00000,7
d2v,0.94898,7
...,...,...
pga,0.00000,5
gmn,0.00000,3
dg2,0.00000,1
cff,0.00000,1


In [81]:
filtered = avg_score_df[(avg_score_df['score'] >= 0.5) &
                        (avg_score_df['num sample'] >= 20)]
filtered

,score,num sample
3sw,0.921875,40
nog,0.891626,29
3t2,0.878086,27
2vz,0.875000,49
pz5,0.875000,41
...,...,...
ite,0.500000,67
6xh,0.500000,39
jak,0.500000,34
sjz,0.500000,22


# select by rec

In [82]:
rec_names = [lig[0:6] for lig in all_lig]
rec_names = set(rec_names)
len(rec_names)

1547

In [83]:
rec_dict = defaultdict(lambda : [])
for lig in tqdm(all_lig):
    rec_name = lig[0:6]
    rec_dict[rec_name].append(lig)

100%|███████████████████████████████████████████████████████████████████| 15075/15075 [00:00<00:00, 1666380.27it/s]


In [84]:
rev_rec_dict = {lig:rec_name for rec_name, ligs in rec_dict.items() for lig in ligs }

In [85]:
scores_by_rec = defaultdict(lambda : [])
for lig, score in tqdm(score_dict.items()):
    rec_name = rev_rec_dict[lig]
    scores_by_rec[rec_name].append(score)

100%|███████████████████████████████████████████████████████████████████| 15075/15075 [00:00<00:00, 1432954.85it/s]


In [86]:
avg_score_rec = {}
n_rec = {}
for rec, scores in tqdm(scores_by_rec.items()):
    avg_score_rec[rec] = np.mean(scores)
    n_rec[rec] = len(scores)

100%|██████████████████████████████████████████████████████████████████████| 1547/1547 [00:00<00:00, 133151.14it/s]


In [87]:
avg_score_rec_df = pd.DataFrame.from_dict({'score':avg_score_rec, 'num sample':n_rec}, orient='columns')
avg_score_rec_df

,score,num sample
1yw8_A,0.327576,5
4f31_B,0.166667,2
5i00_A,0.372994,59
5eob_A,0.423485,39
4ot2_A,0.777778,1
...,...,...
4hyt_B,0.263158,1
5dqf_A,0.142857,1
1goi_A,0.419355,1
1vhw_A,0.222222,1


In [88]:
avg_score_rec_df.sort_values('score', ascending=False, inplace=True)
avg_score_rec_df.to_csv(os.path.join(root_path, fn+'_avg_score_by_rec.csv'))
avg_score_rec_df

,score,num sample
3s0k_A,1.0,1
5jmn_A,1.0,3
3ric_A,1.0,1
2bv7_A,1.0,1
3rzn_A,1.0,1
...,...,...
4u4l_B,0.0,1
5sx8_A,0.0,1
2v2b_A,0.0,1
4gn8_B,0.0,2


In [89]:
filtered_rec = avg_score_rec_df[(avg_score_rec_df['score'] >= 0.6) &
                                (avg_score_rec_df['num sample'] >= 10)]
filtered_rec

,score,num sample
1e3k_A,0.813230,14
4m8h_A,0.809857,39
4rmd_A,0.807615,42
4k4j_A,0.804140,33
4poj_A,0.803903,43
...,...,...
4fk6_B,0.609872,24
4i5c_B,0.607862,23
5wo4_A,0.606769,21
5khx_A,0.602460,20


In [90]:
rec_dict['1c8j_A']

[]